In [4]:
import pandas as pd
from shapely.wkt import loads

# https://data.cityofnewyork.us/City-Government/Parking-tickets/sdcp-g4qp
# df = pd.read_csv("../linearizations/input_files/Parking_Violations_Iussed.csv")
# df

rides_df = pd.read_csv("./taxis.csv", delimiter=";")
zones_df = pd.read_csv("./taxi_zones.csv")

# generate a representative geo points for each taxi ride's pick-up and drop-off locations, so that 
# we can linearize by geospace
representatives = []

for i, zone in enumerate(zones_df["the_geom"]):
  representatives += [str(loads(zone).representative_point())]

zones_df["representative"] = representatives
zones_df = zones_df[["zone", "LocationID", "representative"]].set_index("LocationID")

# for convenience, append the representatives as columns to the rides table via join
zones_df = zones_df.rename(columns={"zone": "PUZone", "representative": "PURepresentative"})
rides_df = rides_df.join(zones_df, on="PULocationID")
zones_df = zones_df.rename(columns={"PUZone": "DOZone", "PURepresentative": "DORepresentative"})
rides_df = rides_df.join(zones_df, on="DOLocationID", rsuffix="DO")
rides_df.head()

,tripID,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PUZone,PURepresentative,DOZone,DORepresentative
0,31852922,2,07/18/2018 06:17:11 AM,07/18/2018 06:23:06 AM,2,2.00,1,N,249,161,...,0.0,0.5,1.66,0.0,0.3,9.96,West Village,POINT (-74.00249693107948 40.73461149649988),Midtown Center,POINT (-73.97743183897771 40.758226398499936)
1,16785706,2,01/09/2018 02:35:11 AM,01/09/2018 02:49:07 AM,1,5.10,5,N,125,265,...,0.0,0.5,15.32,10.5,0.3,76.62,Hudson Sq,POINT (-74.00717582647246 40.72537622752054),NaN,NaN
2,66379394,2,11/24/2018 05:07:34 PM,11/24/2018 05:13:47 PM,2,1.03,1,N,239,238,...,0.0,0.5,0.00,0.0,0.3,6.80,Upper West Side South,POINT (-73.97827330208816 40.784107089122664),Upper West Side North,POINT (-73.97281447968041 40.79176596039123)
3,10428271,2,06/15/2018 08:41:34 AM,06/15/2018 08:47:41 AM,1,0.82,1,N,249,114,...,0.0,0.5,1.70,0.0,0.3,8.50,West Village,POINT (-74.00249693107948 40.73461149649988),Greenwich Village South,POINT (-73.9986779689641 40.7286116144999)
4,23940933,1,02/02/2018 07:43:08 AM,02/02/2018 07:50:18 AM,2,1.80,1,N,239,166,...,0.0,0.5,0.42,0.0,0.3,8.72,Upper West Side South,POINT (-73.97827330208816 40.784107089122664),Morningside Heights,POINT (-73.9618154733028 40.809569690947164)
